In [1]:
!pip install transformers torch

In [2]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
import pandas as pd

In [3]:
!wget https://indiaai.s3.ap-south-1.amazonaws.com/docs/test.csv
#!wget https://indiaai.s3.ap-south-1.amazonaws.com/docs/train.csv  ## upload cleansed train data directly

--2024-11-05 03:46:19--  https://indiaai.s3.ap-south-1.amazonaws.com/docs/test.csv
Resolving indiaai.s3.ap-south-1.amazonaws.com (indiaai.s3.ap-south-1.amazonaws.com)... 52.219.156.90, 52.219.158.150, 3.5.212.157, ...
Connecting to indiaai.s3.ap-south-1.amazonaws.com (indiaai.s3.ap-south-1.amazonaws.com)|52.219.156.90|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14069068 (13M) [text/csv]
Saving to: ‘test.csv’

test.csv            100%[===================>]  13.42M  21.4MB/s    in 0.6s    

2024-11-05 03:46:20 (21.4 MB/s) - ‘test.csv’ saved [14069068/14069068]



In [4]:
df_train = pd.read_csv('/content/train.csv')
df_test = pd.read_csv('/content/test.csv')

In [5]:
df_train.describe()

,category,sub_category,crimeaditionalinfo
count,83437,83437,83437
unique,15,39,77858
top,Online Financial Fraud,UPI Related Frauds,fraud through upi fraud phone pe bank state ba...
freq,50794,22891,195


In [21]:
#df_train.loc[df_train["sub_category"].isnull(), "sub_category"] = df_train["category"]
#df_train= df_train.dropna(how='any')

In [22]:
#df_train.describe()

,category,sub_category,crimeaditionalinfo
count,83437,83437,83437
unique,15,39,77858
top,Online Financial Fraud,UPI Related Frauds,fraud through upi fraud phone pe bank state ba...
freq,50794,22891,195


In [6]:
df_test.describe()

,category,sub_category,crimeaditionalinfo
count,31229,28993,31222
unique,15,37,28785
top,Online Financial Fraud,UPI Related Frauds,Respected Sir\r\n\r\nA very serious matter I w...
freq,18896,8890,753


In [7]:
df_test.loc[df_test["sub_category"].isnull(), "sub_category"] = df_test["category"]
df_test= df_test.dropna(how='any')

In [8]:
df_test.describe()

,category,sub_category,crimeaditionalinfo
count,31222,31222,31222
unique,15,41,28785
top,Online Financial Fraud,UPI Related Frauds,Respected Sir\r\n\r\nA very serious matter I w...
freq,18890,8886,753


In [9]:
sub_categories = sorted(set(df_test["sub_category"].unique().tolist() + df_train["sub_category"].unique().tolist()))
d_cat = {}
for i, cat in enumerate(sub_categories):
  d_cat[cat] = i
d_cat

{'Against Interest of sovereignty or integrity of India': 0,
 'Business Email CompromiseEmail Takeover': 1,
 'Cheating by Impersonation': 2,
 'Child Pornography CPChild Sexual Abuse Material CSAM': 3,
 'Computer Generated CSAM/CSEM': 4,
 'Cryptocurrency Fraud': 5,
 'Cyber Blackmailing & Threatening': 6,
 'Cyber Bullying  Stalking  Sexting': 7,
 'Cyber Terrorism': 8,
 'Damage to computer computer systems etc': 9,
 'Data Breach/Theft': 10,
 'DebitCredit Card FraudSim Swap Fraud': 11,
 'DematDepository Fraud': 12,
 'Denial of Service (DoS)/Distributed Denial of Service (DDOS) attacks': 13,
 'EMail Phishing': 14,
 'EWallet Related Fraud': 15,
 'Email Hacking': 16,
 'FakeImpersonating Profile': 17,
 'Fraud CallVishing': 18,
 'Hacking/Defacement': 19,
 'Impersonating Email': 20,
 'Internet Banking Related Fraud': 21,
 'Intimidating Email': 22,
 'Malware Attack': 23,
 'Online Gambling  Betting': 24,
 'Online Job Fraud': 25,
 'Online Matrimonial Fraud': 26,
 'Online Trafficking': 27,
 'Other':

In [10]:
df_train["subcat_label"] = df_train["sub_category"].apply(lambda x: d_cat.get(x, "unknown"))
df_test["subcat_label"] = df_test["sub_category"].apply(lambda x: d_cat.get(x, "unknown"))

train_text = df_train["crimeaditionalinfo"].tolist()
train_labels = df_train["subcat_label"].tolist()
test_text = df_test["crimeaditionalinfo"].tolist()
test_labels = df_test["subcat_label"].tolist()

In [11]:
sorted(df_test["subcat_label"].unique())

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41]

In [12]:
sorted(df_train["subcat_label"].unique())

[0,
 1,
 2,
 3,
 5,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 36,
 37,
 38,
 39,
 40,
 41]

In [13]:
# Load tokenizer and model
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=42)  # Set `num_labels` to the number of categories
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [14]:
from torch.utils.data import Dataset

# Custom dataset class
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # Tokenize text
        inputs = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt")

        # Return input_ids and attention_mask as well as the label
        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Initialize dataset
train_set = TextClassificationDataset(train_text[:40000], train_labels[:40000], tokenizer) # change to feed full dataset
test_set = TextClassificationDataset(test_text[:8000], test_labels[:8000], tokenizer) # change to feed full dataset

In [15]:
len(train_set), len(test_set)

(40000, 8000)

In [16]:
# Training arguments
batch_size = 4
num_epochs = 2
train_dataset_size = len(train_set)
steps_per_epoch = train_dataset_size // batch_size

training_args = TrainingArguments(
    output_dir="./results",           # output directory
    overwrite_output_dir=True,        # output directory
    num_train_epochs=num_epochs,      # number of epochs
    per_device_train_batch_size=4,    # batch size for training
    per_device_eval_batch_size=4,     # batch size for evaluation
    eval_strategy="steps",            # evaluate every epoch
    logging_dir="./logs",
    logging_steps= 5000, #steps_per_epoch,
    save_strategy="epoch",            # save checkpoints at end of epoch
    save_total_limit=1,               # save only latest check point
    #max_steps=10,                    # Stop training after 10 steps
    report_to="none"                  # Disable W&B logging
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set,
)

# Fine-tune the model
trainer.train()

Step,Training Loss,Validation Loss
5000,1.807100,1.678554
10000,1.585000,1.536832
15000,1.422300,1.533335
20000,1.390300,1.493063


TrainOutput(global_step=20000, training_loss=1.5511707275390625, metrics={'train_runtime': 2834.1107, 'train_samples_per_second': 28.228, 'train_steps_per_second': 7.057, 'total_flos': 5264111001600000.0, 'train_loss': 1.5511707275390625, 'epoch': 2.0})

In [17]:
def predict_category(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = logits.argmax(dim=-1).item()  # Get the index of the highest score
    return predicted_class

# Example prediction
print(f'crime description : {test_text[0]}')
print(f'actual category : {list(d_cat.items())[test_labels[0]]}')
new_text = test_text[2]
predicted_category = predict_category(new_text)
print(f"Predicted category: {list(d_cat.items())[predicted_category]}")

crime description : Sir namaskar  mein Ranjit Kumar PatraPaise nehi the tho sir kuch din pehele online loan aap Credit pearl loan aap se money loan kiya thalekin sir  loan bolke jub  loan diye tho mein turant return kar diya thalekin din baad whats app pe message aya payment karomein bola  diye the aap mein wo de diyawo gali diye tho  v return kar diyafir v message karke bolte hai full payment karo half payment nehi chalegarape case mein daldenge etcFake or illigal se contact number v hack kar dete haibol rahehai sab ko message karenge ye rapist hai bolke sirpls sir small ammount ke liye goggle play store se loan apply kiya thaFake loan aap v hai socha nehi thapls sir request kar rahahun action lo Sir  mera number  hai jo v proof chahiye dunga Sir 
actual category : ('RapeGang Rape RGRSexually Abusive Content', 33)
Predicted category: ('Hacking/Defacement', 19)


In [20]:
pred_labels = []
for text in test_text:
  pred = predict_category(text)
  pred_labels.append(pred)
pred_labels

[28,
 39,
 19,
 28,
 28,
 39,
 28,
 39,
 21,
 11,
 7,
 28,
 7,
 39,
 39,
 18,
 28,
 39,
 39,
 11,
 11,
 39,
 39,
 39,
 39,
 37,
 39,
 19,
 39,
 39,
 39,
 19,
 39,
 16,
 29,
 29,
 11,
 39,
 11,
 19,
 39,
 11,
 28,
 19,
 39,
 28,
 28,
 39,
 17,
 40,
 39,
 39,
 39,
 11,
 11,
 18,
 17,
 16,
 28,
 39,
 21,
 33,
 39,
 7,
 18,
 39,
 11,
 18,
 28,
 25,
 28,
 11,
 39,
 39,
 21,
 11,
 7,
 17,
 28,
 39,
 21,
 19,
 11,
 21,
 39,
 21,
 17,
 19,
 11,
 28,
 19,
 39,
 17,
 11,
 11,
 18,
 33,
 39,
 28,
 7,
 28,
 39,
 21,
 39,
 39,
 18,
 39,
 14,
 33,
 11,
 33,
 21,
 39,
 39,
 28,
 39,
 21,
 15,
 7,
 39,
 39,
 28,
 39,
 39,
 39,
 28,
 11,
 28,
 39,
 28,
 11,
 39,
 39,
 28,
 21,
 39,
 39,
 18,
 28,
 11,
 28,
 16,
 39,
 39,
 39,
 21,
 39,
 39,
 40,
 15,
 39,
 39,
 18,
 28,
 17,
 39,
 39,
 39,
 28,
 39,
 33,
 18,
 28,
 28,
 39,
 19,
 39,
 29,
 39,
 39,
 28,
 39,
 18,
 39,
 21,
 7,
 33,
 40,
 11,
 39,
 39,
 39,
 39,
 11,
 19,
 33,
 28,
 11,
 39,
 39,
 28,
 39,
 33,
 7,
 28,
 11,
 39,
 39,
 18,
 21,
 11,
 28

In [21]:
from sklearn.metrics import accuracy_score, f1_score
accuracy = accuracy_score(test_labels, pred_labels)
f1 = f1_score(test_labels, pred_labels, average="weighted")  # "weighted" accounts for class imbalance

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.5566
F1 Score: 0.5253
